# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather = pd.read_csv("../WeatherPy/Export/City_Weather.csv")
display(cities_weather)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudliness,Wind Speed,Country,Date
0,0,Albany,-64.724715,107.207715,8.71,93.0,100.0,21.54,AU,1.650969e+09
1,1,Progreso,24.641227,-90.523992,77.45,84.0,0.0,14.61,MX,1.650908e+09
2,2,Comodoro rivadavia,-44.148168,-67.638349,54.50,41.0,100.0,20.45,AR,1.650919e+09
3,3,Luderitz,-31.317895,0.758280,65.55,67.0,100.0,14.90,NaN,1.650929e+09
4,4,Yangambi,-0.071054,23.789763,69.53,99.0,88.0,2.04,CD,1.650937e+09
...,...,...,...,...,...,...,...,...,...,...
603,603,Kalutara,6.663162,79.969598,76.41,87.0,69.0,3.47,LK,1.650950e+09
604,604,Sirnak,37.535885,42.473748,48.20,70.0,63.0,3.58,TR,1.650941e+09
605,605,Zhigansk,70.032623,124.432684,7.97,90.0,61.0,3.02,RU,1.650962e+09
606,606,Iringa,-8.174751,35.825309,57.06,93.0,96.0,4.21,TZ,1.650941e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_weather[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidities = cities_weather["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
warm_cities = cities_weather[cities_weather["Max Temp"] > 70]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = warm_cities[["City", "Lat", "Lng", "Country"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Country,Hotel Name
1,Progreso,24.641227,-90.523992,MX,
7,Tigre,10.768338,-78.310027,PA,
8,Beroroha,-21.269559,44.263833,MG,
15,Boende,-1.866959,20.661703,CD,
16,Englewood,26.032704,-83.239558,US,
...,...,...,...,...,...
592,Chirongui,-13.580934,44.945361,YT,
593,Kumhari,21.462311,81.367146,IN,
595,Puro,17.929262,119.310242,PH,
598,Kijang,0.391396,106.357806,ID,


In [9]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # find the closest hotel to coordinates
    coordinates = (str(row["Lat"]) + "," + str(row["Lng"]))
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = coordinates
    
    print("Retrieving Results for " + row["City"])
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print("Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
#         hotel_df.loc[index, 'address'] = results[0]['vicinity']
#         hotel_df.loc[index, 'price_level'] = results[0]['price_level']
#         hotel_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Progreso
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Tigre
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Beroroha
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Boende
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Englewood
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Faanui
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Jian
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Ixtapa
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Sao felix do xingu
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Makakilo city
Closest hotel is {re

Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Elat
Closest hotel is {results[0]['name']}.
Retrieving Results for Wenling
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Grand baie
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Hambantota
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Hurghada
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Elizabeth city
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Sardarshahr
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Lahat
Closest hotel is {results[0]['name']}.
Retrieving Results for Pemangkat
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Cap malheureux
Closes

Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for George town
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Richards bay
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Jacqueville
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Bima
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Pozo colorado
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Sal rei
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Bambanglipuro
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Faya
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Bathsheba
Closest hotel is {results[0]['name']}.
Miss

Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Manacapuru
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Mamou
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Tres palos
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Merauke
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Fare
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for High rock
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Acajutla
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Chirongui
Closest hotel is {results[0]['name']}.
Missing field/result... skipping.
Retrieving Results for Kumhari
Closest hotel is {results[0]['name']}.
Retrieving Results 

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))